In [1]:
from eos import projroot

Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.pool.pool.Pool[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): <class 'pandas.core.frame.DataFrame'>; Concrete Class: (eos.data_io.pool.pool.Pool[pandas.core.frame.DataFrame],)
Pool.__init_subclass__(): <class 'eos.data_io.struct.eos_data.RecordDoc'>; Concrete Class: (eos.data_io.pool.pool.Pool[eos.data_io.struct.eos_data.RecordDoc],)
Pool.__init_subclass__(): <class 'pandas.core.frame.DataFrame'>; Concrete Class: (eos.data_io.pool.pool.Pool[pandas.core.frame.DataFrame],)
Pool.__init_subclass__(): <class 'pandas.core.frame.DataFrame'>; Concrete Class: (eos.data_io.pool.pool.Pool[pandas.core.frame.DataFrame],)
Pool.__init_subclass__(): <class 'pandas.core.frame.DataFrame'>; Concrete Class: (eos.data_io.pool.pool.Pool[pandas.core.frame.DataFrame],)
Pool.__init_subclass__(): ~DocItemT; Concrete Class: (eos.data_io.pool.pool.Pool[~DocItemT], typing.Generic[~DocItemT])
Pool.__init_subclass__(): ~DocItemT; Con

2023-07-21 13:41:48.570687: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-21 13:41:48.605412: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-21 13:41:48.605952: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Pool.__init_subclass__(): <class 'pandas.core.frame.DataFrame'>; Concrete Class: (eos.data_io.buffer.buffer.Buffer[pandas.core.frame.DataFrame],)
Pool.__init_subclass__(): <class 'pandas.core.frame.DataFrame'>; Concrete Class: (eos.data_io.buffer.buffer.Buffer[pandas.core.frame.DataFrame],)


In [2]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from fastavro import writer, reader, parse_schema


In [3]:
import os
os.chdir(projroot / 'data')
os.getcwd()

'/home/n/devel/anting_remote/eos/data'

In [4]:
from dataclasses import dataclass, field
from functools import reduce

from typing import Optional
import pandas as pd
import numpy as np
from ordered_set import OrderedSet
from eos.data_io.config import drivers, trucks_by_id, drivers_by_id
from datetime import datetime

a = np.array([[1, 2, 3], [4, 5, 6]])
ts = pd.to_datetime(datetime.now())


ts_ind = ts + pd.to_timedelta(np.arange(0, 4 * 20, 20), 'ms')
ss = np.arange(12)
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]
df_ss = pd.DataFrame(
    {'timestep': ts_ind, 'velocity': a1, 'thrust': a2, 'brake': a3}
)  # .set_index('timestep')
df_ss.columns.name = 'qtuple'

state = df_ss.stack().swaplevel(0, 1)
state.name = 'state'
state.index.names = ['rows', 'idx']
state.sort_index(inplace=True)

a = len(ss) + np.arange(15)
speed_ser = pd.Series(np.linspace(40, 60, 3), name='speed')
row_array = a.reshape(3, 5).transpose()
rows_df = pd.DataFrame(row_array)
rows_df.columns = [f'r{i}' for i in np.arange(3)]

ts_ind = ts + pd.to_timedelta(np.arange(5 * 20, 8 * 20, 20), 'ms')
ts_ser = pd.Series(ts_ind, name='timestep')
throttle_ser = pd.Series(np.linspace(0, 1.0, 5), name='throttle')
# throttle_ser
dfs = [rows_df, ts_ser, speed_ser, throttle_ser]
action = (
    reduce(
        lambda left, right: pd.merge(
            left, right, how='outer', left_index=True, right_index=True
        ),
        dfs,
    )
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)

action.name = 'action'
action.index.names = ['rows', 'idx']

reward = (
    pd.DataFrame({'work': len(ss) + len(a), 'timestep': ts_ind[0]}, index=[0])
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)
reward.index.names = ['rows', 'idx']
reward.name = 'reward'
ts_ind = ts + pd.to_timedelta(5, 's') + pd.to_timedelta(np.arange(0, 4 * 20, 20), 'ms')
ss = (
    np.arange(12) + len(ss) + len(a) + len(reward) - 1
)  # exclude the timestamp in reward
a1 = ss[:4]
a2 = ss[4:8]
a3 = ss[8:]

nstate = (
    pd.DataFrame({'timestep': ts_ind, 'velocity': a1, 'thrust': a2, 'brake': a3})
    .stack()
    .swaplevel(0, 1)
    .sort_index()
)
nstate.name = 'nstate'
nstate.index.names = ['rows', 'idx']

timestamp = pd.Series([ts], name='timestamp')
timestamp.index = pd.MultiIndex.from_product(
    [timestamp.index, [0]], names=['rows', 'idx']
)

timestamp_index = (timestamp.name, '', 0)
state_index = [(state.name, *i) for i in state.index]
reward_index = [(reward.name, *i) for i in reward.index]
action_index = [(action.name, *i) for i in action.index]
nstate_index = [(nstate.name, *i) for i in nstate.index]
multiindex = pd.MultiIndex.from_tuples(
    [timestamp_index, *state_index, *action_index, *reward_index, *nstate_index]
)
observation_list = [timestamp, state, action, reward, nstate]
observation = pd.concat(observation_list)
observation.index = multiindex

observation0 = observation.copy()
observation0.loc['timestamp', '', 0] = ts + pd.Timedelta(1, 'h')
observation1 = observation.copy()
observation1.loc['timestamp', '', 0] = ts + pd.Timedelta(2, 'h')
observation2 = observation.copy()
observation2.loc['timestamp', '', 0] = ts + pd.Timedelta(3, 'h')
observation3 = observation.copy()
observation3.loc['timestamp', '', 0] = ts + pd.Timedelta(4, 'h')
observation4 = observation.copy()
observation4.loc['timestamp', '', 0] = ts + pd.Timedelta(5, 'h')
observation_list = [
    observation0,
    observation1,
    observation2,
    observation3,
    observation4,
]

dfs_epi = pd.concat(observation_list, axis=1).transpose()
dfs_epi.columns.names = ['qtuple', 'rows', 'idx']

dfs_episode = dfs_epi.copy()
dfs_episode.set_index(('timestamp', '', 0), inplace=True)
dfs_episode.sort_index(axis=1, inplace=True)
dfs_episode.index.name = 'timestamp'

state_cols_float = [('state', col) for col in ['brake', 'thrust', 'velocity']]
action_cols_float = [('action', col) for col in ['r0', 'r1', 'r2', 'speed', 'throttle']]
reward_cols_float = [('reward', 'work')]
nstate_cols_float = [('nstate', col) for col in ['brake', 'thrust', 'velocity']]
for col in action_cols_float + state_cols_float + reward_cols_float + nstate_cols_float:
    dfs_episode[col[0], col[1]] = dfs_episode[col[0], col[1]].astype(
        'float'
    )  # float16 not allowed in parquet
dfs_episode = pd.concat(
    [dfs_episode], keys=[drivers_by_id['wang-cheng'].pid], names=['driver']
)
dfs_episode = pd.concat(
    [dfs_episode], keys=[trucks_by_id['VB7'].vid], names=['vehicle']
)
dfs_episode = pd.concat([dfs_episode], keys=[ts], names=['episodestart'])
dfs_episode = dfs_episode.swaplevel(1, 0, axis=0)
dfs_episode = dfs_episode.swaplevel(1, 2, axis=0)
dfs_episode.sort_index(inplace=True)
dfs_episode

qtuple                                                                   action  \
rows                                                                         r0   
idx                                                                           0   
vehicle driver     episodestart               timestamp                           
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535   12.0   
                                              2023-07-21 15:41:52.277535   12.0   
                                              2023-07-21 16:41:52.277535   12.0   
                                              2023-07-21 17:41:52.277535   12.0   
                                              2023-07-21 18:41:52.277535   12.0   

qtuple                                                                          \
rows                                                                             
idx                                                                          1   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  13.0   
                                              2023-07-21 15:41:52.277535  13.0   
                                              2023-07-21 16:41:52.277535  13.0   
                                              2023-07-21 17:41:52.277535  13.0   
                                              2023-07-21 18:41:52.277535  13.0   

qtuple                                                                          \
rows                                                                             
idx                                                                          2   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  14.0   
                                              2023-07-21 15:41:52.277535  14.0   
                                              2023-07-21 16:41:52.277535  14.0   
                                              2023-07-21 17:41:52.277535  14.0   
                                              2023-07-21 18:41:52.277535  14.0   

qtuple                                                                          \
rows                                                                             
idx                                                                          3   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  15.0   
                                              2023-07-21 15:41:52.277535  15.0   
                                              2023-07-21 16:41:52.277535  15.0   
                                              2023-07-21 17:41:52.277535  15.0   
                                              2023-07-21 18:41:52.277535  15.0   

qtuple                                                                          \
rows                                                                             
idx                                                                          4   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  16.0   
                                              2023-07-21 15:41:52.277535  16.0   
                                              2023-07-21 16:41:52.277535  16.0   
                                              2023-07-21 17:41:52.277535  16.0   
                                              2023-07-21 18:41:52.277535  16.0   

qtuple                                                                          \
rows                                                                        r1   
idx                                                                          0   
vehicle driver     episodestart               time

In [5]:
ts_new = pd.to_datetime((datetime.now()))
episodestart = ts_new - pd.Timedelta(2, 'd')

dfs_episode0 = dfs_episode.copy()
dfs_episode0.index = dfs_episode0.index.set_levels([episodestart], level='episodestart')
dfs_episode0.index = dfs_episode0.index.set_levels(
    [[trucks_by_id['VB7'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
ts_index = dfs_episode0.index.unique(level='timestamp')
idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num-1)
ts_index_to_drop = np.random.choice(ts_index, drop_num,replace=False)
dfs_episode0 =dfs_episode0.drop(index=ts_index_to_drop, level='timestamp')
# srs_episode0 = dfs_episode0.stack(level=['qtuple', 'rows', 'idx'])

dfs_episode1 = dfs_episode.copy()
dfs_episode1.index = dfs_episode1.index.set_levels(
    [episodestart - pd.Timedelta(3, 'd')], level='episodestart'
)
dfs_episode1.index = dfs_episode1.index.set_levels(
    [[trucks_by_id['MP73'].vid], [drivers_by_id['wang-cheng'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
ts_index = dfs_episode1.index.unique(level='timestamp')
idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num-1)
ts_index_to_drop = np.random.choice(ts_index, drop_num,replace=False)
dfs_episode1 =dfs_episode1.drop(index=ts_index_to_drop, level='timestamp')
# srs_episode1 = dfs_episode1.stack(level=['qtuple', 'rows', 'idx'])

dfs_episode2 = dfs_episode.copy()
dfs_episode2.index = dfs_episode2.index.set_levels(
    [episodestart - pd.Timedelta(4, 'd')], level='episodestart'
)
dfs_episode2.index = dfs_episode2.index.set_levels(
    [[trucks_by_id['VB7'].vid], [drivers_by_id['wang-cheng'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
ts_index = dfs_episode2.index.unique(level='timestamp')
idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num-1)
ts_index_to_drop = np.random.choice(ts_index, drop_num,replace=False)
dfs_episode2 =dfs_episode2.drop(index=ts_index_to_drop, level='timestamp')
# srs_episode2 = dfs_episode2.stack(level=['qtuple', 'rows', 'idx'])

dfs_episode3 = dfs_episode.copy()
dfs_episode3.index = dfs_episode3.index.set_levels(
    [episodestart - pd.Timedelta(5, 'd')], level='episodestart'
)
dfs_episode3.index = dfs_episode3.index.set_levels(
    [[trucks_by_id['MP73'].vid], [drivers_by_id['zheng-longfei'].pid]],
    level=['vehicle', 'driver'],
    verify_integrity=False,
)
ts_index = dfs_episode3.index.unique(level='timestamp')
idx_num = len(ts_index)
drop_num = np.random.randint(low=1, high=idx_num-1)
ts_index_to_drop = np.random.choice(ts_index, drop_num,replace=False)
dfs_episode3 =dfs_episode3.drop(index=ts_index_to_drop, level='timestamp')
# srs_episode3 = dfs_episode3.stack(level=['qtuple', 'rows', 'idx'])
from functools import reduce

episodes = [dfs_episode, dfs_episode0, dfs_episode1, dfs_episode2, dfs_episode3]
try:
    dfs_episode_all = reduce(
        lambda left, right,: pd.concat([left, right], axis=0), episodes
    )
    dfs_episode_all.sort_index(inplace=True)
    dfs_episode_all = dfs_episode_all[['state', 'action', 'reward', 'nstate']]
    dfs_episode_all

except Exception as e:
    print(e)
# dfs_episode_all.sort_index(inplace=True)
# dfs_episode_all = dfs_episode_all.swaplevel(1, 0, axis=0)
# dfs_episode_all = dfs_episode_all.swaplevel(1, 2, axis=0)
# dfs_episode_all.index
# dfs_episode_all.columns#
#

qtuple                                                                      state  \
rows                                                                        brake   
idx                                                                             0   
vehicle driver        episodestart               timestamp                          
MP73    wang-cheng    2023-07-16 13:41:52.367182 2023-07-21 15:41:52.277535   8.0   
                                                 2023-07-21 18:41:52.277535   8.0   
        zheng-longfei 2023-07-14 13:41:52.367182 2023-07-21 15:41:52.277535   8.0   
                                                 2023-07-21 18:41:52.277535   8.0   
VB7     wang-cheng    2023-07-15 13:41:52.367182 2023-07-21 14:41:52.277535   8.0   
                                                 2023-07-21 16:41:52.277535   8.0   
                                                 2023-07-21 17:41:52.277535   8.0   
                      2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535   8.0   
                                                 2023-07-21 15:41:52.277535   8.0   
                                                 2023-07-21 16:41:52.277535   8.0   
                                                 2023-07-21 17:41:52.277535   8.0   
                                                 2023-07-21 18:41:52.277535   8.0   
        zheng-longfei 2023-07-19 13:41:52.367182 2023-07-21 14:41:52.277535   8.0   
                                                 2023-07-21 15:41:52.277535   8.0   
                                                 2023-07-21 17:41:52.277535   8.0   
                                                 2023-07-21 18:41:52.277535   8.0   

qtuple                                                                            \
rows                                                                               
idx                                                                            1   
vehicle driver        episodestart               timestamp                         
MP73    wang-cheng    2023-07-16 13:41:52.367182 2023-07-21 15:41:52.277535  9.0   
                                                 2023-07-21 18:41:52.277535  9.0   
        zheng-longfei 2023-07-14 13:41:52.367182 2023-07-21 15:41:52.277535  9.0   
                                                 2023-07-21 18:41:52.277535  9.0   
VB7     wang-cheng    2023-07-15 13:41:52.367182 2023-07-21 14:41:52.277535  9.0   
                                                 2023-07-21 16:41:52.277535  9.0   
                                                 2023-07-21 17:41:52.277535  9.0   
                      2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  9.0   
                                                 2023-07-21 15:41:52.277535  9.0   
                                                 2023-07-21 16:41:52.277535  9.0   
                                                 2023-07-21 17:41:52.277535  9.0   
                                                 2023-07-21 18:41:52.277535  9.0   
        zheng-longfei 2023-07-19 13:41:52.367182 2023-07-21 14:41:52.277535  9.0   
                                                 2023-07-21 15:41:52.277535  9.0   
                                                 2023-07-21 17:41:52.277535  9.0   
                                                 2023-07-21 18:41:52.277535  9.0   

qtuple                                                                             \
rows                                                                                
idx                                                                             2   
vehicle driver        episodestart               timestamp                          
MP73    wang-cheng    2023-07-16 13:41:52.367182 2023-07-21 15:41:52.277535  10.0   
                                                 2023-07-21 18:41:52.277535  10.0   
        zheng-longfei 2023-07-14 13:41:52.367182 2023-07-21 15:41:52.277535  10.0   
                       

In [6]:
dfs_episode
# episodes[0]

qtuple                                                                   action  \
rows                                                                         r0   
idx                                                                           0   
vehicle driver     episodestart               timestamp                           
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535   12.0   
                                              2023-07-21 15:41:52.277535   12.0   
                                              2023-07-21 16:41:52.277535   12.0   
                                              2023-07-21 17:41:52.277535   12.0   
                                              2023-07-21 18:41:52.277535   12.0   

qtuple                                                                          \
rows                                                                             
idx                                                                          1   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  13.0   
                                              2023-07-21 15:41:52.277535  13.0   
                                              2023-07-21 16:41:52.277535  13.0   
                                              2023-07-21 17:41:52.277535  13.0   
                                              2023-07-21 18:41:52.277535  13.0   

qtuple                                                                          \
rows                                                                             
idx                                                                          2   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  14.0   
                                              2023-07-21 15:41:52.277535  14.0   
                                              2023-07-21 16:41:52.277535  14.0   
                                              2023-07-21 17:41:52.277535  14.0   
                                              2023-07-21 18:41:52.277535  14.0   

qtuple                                                                          \
rows                                                                             
idx                                                                          3   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  15.0   
                                              2023-07-21 15:41:52.277535  15.0   
                                              2023-07-21 16:41:52.277535  15.0   
                                              2023-07-21 17:41:52.277535  15.0   
                                              2023-07-21 18:41:52.277535  15.0   

qtuple                                                                          \
rows                                                                             
idx                                                                          4   
vehicle driver     episodestart               timestamp                          
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  16.0   
                                              2023-07-21 15:41:52.277535  16.0   
                                              2023-07-21 16:41:52.277535  16.0   
                                              2023-07-21 17:41:52.277535  16.0   
                                              2023-07-21 18:41:52.277535  16.0   

qtuple                                                                          \
rows                                                                        r1   
idx                                                                          0   
vehicle driver     episodestart               time

In [7]:
from dataclasses import asdict
indices = dfs_episode.index
# ep_start = indices.get_level_values(level='episodestart')[0]
indices_dict = [
    {indices.names[i]: level for i, level in enumerate(levels)} for levels in indices
]
episode_meta = indices_dict[0].copy()
episode_meta['episodestart'] = episode_meta['episodestart'].timestamp()*1e6
episode_meta['timestamp'] = episode_meta['timestamp'].timestamp()*1e6
try:
    episode_meta.pop('timestamp')
except KeyError:
    print(f"Key 'timestamp' not found")
# episode_meta

# episodes_indices
episodes_indices_dict = [
    [{indices.names[i]: level for i, level in enumerate(levels)} for levels in df.index]
    for df in episodes
]
for indices in episodes_indices_dict:
    indices[0].pop('timestamp')

episodes_meta_dict = [indices[0] for indices in episodes_indices_dict]

for ep_mt in episodes_meta_dict:
    ep_mt['episodestart'] = ep_mt['episodestart'].timestamp()*1e6
episodes_meta_dict

1689950512277535.0

Timestamp('2023-07-21 14:41:52.277535')

Timestamp('2023-07-21 14:41:52.277535')

Timestamp('2023-07-21 15:41:52.277535')

Timestamp('2023-07-21 14:41:52.277535')

Timestamp('2023-07-21 15:41:52.277535')

[{'vehicle': 'VB7',
  'driver': 'wang-cheng',
  'episodestart': 1689946912277535.0},
 {'vehicle': 'VB7',
  'driver': 'zheng-longfei',
  'episodestart': 1689774112367182.0},
 {'vehicle': 'MP73',
  'driver': 'wang-cheng',
  'episodestart': 1689514912367182.0},
 {'vehicle': 'VB7',
  'driver': 'wang-cheng',
  'episodestart': 1689428512367182.0},
 {'vehicle': 'MP73',
  'driver': 'zheng-longfei',
  'episodestart': 1689342112367182.0}]

In [8]:
from eos.data_io.struct import ObservationMeta, StateSpecs, StateUnitCodes, ActionSpecs
from eos.utils.eos_pandas import ep_nest, df_to_ep_nested_dict, avro_ep_encoding
from pydantic import schema_json_of, schema_of

observation_meta = ObservationMeta(
    state_specs=StateSpecs(
        state_unit_codes=StateUnitCodes(
            velocity_unit_code='kph',
            thrust_unit_code='pct',
            brake_unit_code='pct',
        ),
        state_number=4,
        unit_number=trucks_by_id['VB7'].cloud_unit_number,  # 4
        unit_duration=trucks_by_id['VB7'].cloud_unit_duration,  # 1s
        frequency=trucks_by_id['VB7'].cloud_signal_frequency,  # 50 hz
    ),
    action_specs=ActionSpecs(
        action_unit_code='nm',
        action_row_number=trucks_by_id['VB7'].torque_table_row_num_flash,
        action_column_number=len(trucks_by_id['VB7'].pedal_scale),
    ),
    reward_specs={
        'reward_unit': 'wh',
    },
    site=trucks_by_id['VB7'].site,
)




dict_nested = avro_ep_encoding(dfs_episode)
dict_nested

[{'timestamp': 1689950512277535.0,
  'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
   'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
   'r2': [22.0, 23.0, 24.0, 25.0, 26.0],
   'speed': [40.0, 50.0, 60.0],
   'throttle': [0.0, 0.25, 0.5, 0.75, 1.0],
   'timestep': [1689946912377535.0, 1689946912397535.0, 1689946912417535.0]},
  'nstate': {'brake': [36.0, 37.0, 38.0, 39.0],
   'thrust': [32.0, 33.0, 34.0, 35.0],
   'timestep': [1689946917277535.0,
    1689946917297535.0,
    1689946917317535.0,
    1689946917337535.0],
   'velocity': [28.0, 29.0, 30.0, 31.0]},
  'reward': {'timestep': [1689946912377535.0], 'work': [27.0]},
  'state': {'brake': [8.0, 9.0, 10.0, 11.0],
   'thrust': [4.0, 5.0, 6.0, 7.0],
   'timestep': [1689946912277535.0,
    1689946912297535.0,
    1689946912317535.0,
    1689946912337535.0],
   'velocity': [0.0, 1.0, 2.0, 3.0]}},
 {'timestamp': 1689954112277535.0,
  'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
   'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
   'r2': [22.0, 23.0,

In [9]:
dict_ep = {'episodestart': episode_meta['episodestart'],
            'meta': {'episode_meta': episode_meta, 'observation_meta': observation_meta.dict()},
            'sequence': dict_nested}
dict_ep


/tmp/ipykernel_1117621/3977804681.py:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.0.3/migration/
  'meta': {'episode_meta': episode_meta, 'observation_meta': observation_meta.dict()},


{'episodestart': 1689946912277535.0,
 'meta': {'episode_meta': {'vehicle': 'VB7',
   'driver': 'wang-cheng',
   'episodestart': 1689946912277535.0},
  'observation_meta': {'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
     'thrust_unit_code': 'pct',
     'brake_unit_code': 'pct'},
    'state_number': 4,
    'unit_number': 4,
    'unit_duration': 1,
    'frequency': 50},
   'action_specs': {'action_unit_code': 'nm',
    'action_row_number': 4,
    'action_column_number': 17},
   'reward_specs': {'reward_unit': 'wh'},
   'site': 'anting'}},
 'sequence': [{'timestamp': 1689950512277535.0,
   'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
    'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
    'r2': [22.0, 23.0, 24.0, 25.0, 26.0],
    'speed': [40.0, 50.0, 60.0],
    'throttle': [0.0, 0.25, 0.5, 0.75, 1.0],
    'timestep': [1689946912377535.0, 1689946912397535.0, 1689946912417535.0]},
   'nstate': {'brake': [36.0, 37.0, 38.0, 39.0],
    'thrust': [32.0, 33.0, 34.0, 35.0],
    '

In [10]:

episodes_dict_nested = [avro_ep_encoding(ep) for ep in episodes]
episodes_dict_nested[0]

[{'timestamp': 1689950512277535.0,
  'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
   'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
   'r2': [22.0, 23.0, 24.0, 25.0, 26.0],
   'speed': [40.0, 50.0, 60.0],
   'throttle': [0.0, 0.25, 0.5, 0.75, 1.0],
   'timestep': [1689946912377535.0, 1689946912397535.0, 1689946912417535.0]},
  'nstate': {'brake': [36.0, 37.0, 38.0, 39.0],
   'thrust': [32.0, 33.0, 34.0, 35.0],
   'timestep': [1689946917277535.0,
    1689946917297535.0,
    1689946917317535.0,
    1689946917337535.0],
   'velocity': [28.0, 29.0, 30.0, 31.0]},
  'reward': {'timestep': [1689946912377535.0], 'work': [27.0]},
  'state': {'brake': [8.0, 9.0, 10.0, 11.0],
   'thrust': [4.0, 5.0, 6.0, 7.0],
   'timestep': [1689946912277535.0,
    1689946912297535.0,
    1689946912317535.0,
    1689946912337535.0],
   'velocity': [0.0, 1.0, 2.0, 3.0]}},
 {'timestamp': 1689954112277535.0,
  'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
   'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
   'r2': [22.0, 23.0,

In [11]:
dict_nested_states = [[step['state'] for step in ep] for ep in episodes_dict_nested]
dict_nested_nstates = [[step['nstate'] for step in ep] for ep in episodes_dict_nested]
dict_nested_rewards = [[step['reward'] for step in ep] for ep in episodes_dict_nested]
dict_nested_actions = [[step['action'] for step in ep] for ep in episodes_dict_nested]
dict_nested_timestamps = [[step['timestamp'] for step in ep] for ep in episodes_dict_nested]

arr_states = [
    [
        {'ts': ts,
        'state': state_arr[i]
        }
    for i, ts in enumerate(ts_arr)] for (ts_arr, state_arr) in zip(dict_nested_timestamps, dict_nested_states)]
arr_states[0]
# dict_nested_timestamps[0]

[{'ts': 1689950512277535.0,
  'state': {'brake': [8.0, 9.0, 10.0, 11.0],
   'thrust': [4.0, 5.0, 6.0, 7.0],
   'timestep': [1689946912277535.0,
    1689946912297535.0,
    1689946912317535.0,
    1689946912337535.0],
   'velocity': [0.0, 1.0, 2.0, 3.0]}},
 {'ts': 1689954112277535.0,
  'state': {'brake': [8.0, 9.0, 10.0, 11.0],
   'thrust': [4.0, 5.0, 6.0, 7.0],
   'timestep': [1689946912277535.0,
    1689946912297535.0,
    1689946912317535.0,
    1689946912337535.0],
   'velocity': [0.0, 1.0, 2.0, 3.0]}},
 {'ts': 1689957712277535.0,
  'state': {'brake': [8.0, 9.0, 10.0, 11.0],
   'thrust': [4.0, 5.0, 6.0, 7.0],
   'timestep': [1689946912277535.0,
    1689946912297535.0,
    1689946912317535.0,
    1689946912337535.0],
   'velocity': [0.0, 1.0, 2.0, 3.0]}},
 {'ts': 1689961312277535.0,
  'state': {'brake': [8.0, 9.0, 10.0, 11.0],
   'thrust': [4.0, 5.0, 6.0, 7.0],
   'timestep': [1689946912277535.0,
    1689946912297535.0,
    1689946912317535.0,
    1689946912337535.0],
   'velocity': 

In [12]:
import fastavro
import json

state_unit_fields_schema = [{'name': 'velocity_unit_code', 'type': 'string'},
                            {'name': 'thrust_unit_code', 'type': 'string'},
                            {'name': 'brake_unit_code', 'type': 'string'}]
state_specs_fields_schema = [{'name': 'state_unit_codes',
                            'type': {
                                'type': 'record',
                                'name': 'state_unit_codes_',
                                'fields': state_unit_fields_schema
                                },
                            },
                            {'name': 'state_number', 'type': 'int'},
                            {'name': 'unit_number', 'type': 'int'},
                            {'name': 'unit_duration', 'type': 'int'},
                            {'name': 'frequency', 'type': 'int'}]
action_specs_fields_schema = [{'name': 'action_unit_code', 'type': 'string'},
                            {'name': 'action_row_number', 'type': 'int'},
                            {'name': 'action_column_number', 'type': 'int'}]
reward_specs_fields_schema = [{'name': 'reward_unit', 'type': 'string'}]

episode_meta_fields_schema = [{'name': 'vehicle', 'type': 'string'},
                                {'name': 'driver', 'type': 'string'},
                                {'name': 'episodestart', 'type': 'long', 'logicalType': 'timestamp-micros'}]
observation_meta_fields_schema = [
                                {
                                    'name': 'state_specs',
                                    'type': {
                                        'type': 'record',
                                        'name': 'state_specs_',
                                        'fields': state_specs_fields_schema
                                    }
                                },
                                {
                                    'name': 'action_specs',
                                    'type': {
                                        'type': 'record',
                                        'name': 'action_specs_',
                                        'fields': action_specs_fields_schema
                                    }
                                },
                                {
                                    'name': 'reward_specs',
                                    'type': {
                                        'type': 'record',
                                        'name': 'reward_specs_',
                                        'fields': reward_specs_fields_schema
                                    }
                                },
                                {
                                    'name': 'site',
                                    'type': 'string'
                                },
                            ] 

state_fields_schema = [
    {
        'name': 'velocity',
        'type': {
            'type': 'array',
            'items': 'float'
        }
    },
    {
        'name': 'thrust', 
        'type': {
            'type': 'array',
            'items': 'float'
            }
    },
    {
        'name': 'brake', 
        'type': {
            'type': 'array',
            'items': 'float'
        }
    },
    {
        'name': 'timestep', 
        'type': {
            'type': 'array',
            'items': {'type': 'long', 'logicalType': 'timestamp-micros'}
        }
    },
]

torque_table_row_names = ['r0', 'r1', 'r2']
action_fields_schema = [{'name': r, 'type': { 'type': 'array', 'items': 'float'}}  for r in torque_table_row_names]
action_fields_schema += [{'name': 'speed', 'type': { 'type': 'array', 'items': 'float'}},
                        {'name': 'throttle', 'type': { 'type': 'array', 'items': 'float'}},
                        {'name': 'timestep', 'type': { 'type': 'array', 'items': {'type': 'long', 'logicalType': 'timestamp-micros'}}}]


reward_fields_schema = [
    {
        'name': 'work',
        'type': {
            'type': 'array',
            'items': 'float'
            }
    },
    {
        'name': 'timestep',
        'type': {
            'type': 'array',
            'items': {'type': 'long', 'logicalType': 'timestamp-micros'}
        }
    }
]



episode_array_fields_schema = [   
    {
        'type': 'long', 
        'name': 'timestamp',
        'logicalType': 'timestamp-micros',
    },
    {
        'name': 'state',
        'type': {
            'type': 'record',
            'name': 'state_',
            'fields': state_fields_schema,
        },
    },
    {
        'name': 'action',
        'type': {
            'type': 'record',
            'name': 'action_',
            'fields': action_fields_schema,
        },
    },
    {
        'name':'reward',
        'type': {
            'type': 'record',
            'name': 'reward_',
            'fields': reward_fields_schema,
        }
    },
    {
        'name': 'nstate',
        'type': {
            'type': 'record',
            'name': 'nstate_',
            'fields': state_fields_schema,
        },
    },
]

schema_episode = {
    'type': 'record',
    'name': 'episode',
    'doc': 'episode data with a timestamp, meta description and an array of episode steps',
    'fields': [
        {
            'type': 'long',
            'name': 'episodestart',
            'logicalType': 'timestamp-micros'
        },
        {
            'name': 'meta',
            'type': {
                'type': 'record', 'name': 'meta_', 'fields': 
                [
                    {
                        'name': 'episode_meta',
                        'type': {
                            'type': 'record',
                            'name': 'episode_meta_',
                            'fields': episode_meta_fields_schema,
                        }
                    },
                    {
                        'name': 'observation_meta',
                        'type': {
                            'type': 'record',
                            'name': 'observation_meta_',
                            'fields': observation_meta_fields_schema
                        }
                    }
                ]
            }
        },
        {
            'name': 'sequence',
            'type': {
                'type': 'array', 
                'items': {
                    'name': 'step',  # not used in constructing the episode observation array data
                    'type': 'record',
                    'fields': episode_array_fields_schema,
                }
            }
        }
    ]
}

parsed_schema_episode = fastavro.schema.parse_schema(schema_episode)
print(json.dumps(schema_episode, indent=2))


{
  "type": "record",
  "name": "episode",
  "doc": "episode data with a timestamp, meta description and an array of episode steps",
  "fields": [
    {
      "type": "long",
      "name": "episodestart",
      "logicalType": "timestamp-micros"
    },
    {
      "name": "meta",
      "type": {
        "type": "record",
        "name": "meta_",
        "fields": [
          {
            "name": "episode_meta",
            "type": {
              "type": "record",
              "name": "episode_meta_",
              "fields": [
                {
                  "name": "vehicle",
                  "type": "string"
                },
                {
                  "name": "driver",
                  "type": "string"
                },
                {
                  "name": "episodestart",
                  "type": "long",
                  "logicalType": "timestamp-micros"
                }
              ]
            }
          },
          {
            "name": "observati

In [13]:
# records_episodes = [
#     {u'episodestart': episodes_indices_dict[0]['episodestart'],
#      u'meta': {'episode_meta': episode_meta, 'observation_meta': observation_meta.dict()},
#      u'sequence': episodes_dict_nested[0]},
#     {u'episodestart': episodes_indices_dict[1]['episodestart'],
#      u'meta': {'episode_meta': episode_meta, 'observation_meta': observation_meta.dict()},
#      u'sequence': episodes_dict_nested[1]},
#     {u'episodestart': episodes_indices_dict[2]['episodestart'],
#      u'meta': {'episode_meta': episode_meta, 'observation_meta': observation_meta.dict()},
#      u'sequence': episodes_dict_nested[2]},
#     {u'episodestart': episodes_indices_dict[3]['episodestart'],
#      u'meta': {'episode_meta': episode_meta, 'observation_meta': observation_meta.dict()},
#      u'sequence': episodes_dict_nested[3]},
#     {u'episodestart': episodes_indices_dict[4]['episodestart'],
#      u'meta': {'episode_meta': episode_meta, 'observation_meta': observation_meta.dict()},
#      u'sequence': episodes_dict_nested[4]},
# ]


records_episodes = [{'episodestart': episodes_meta_dict[i]['episodestart'],
                    'meta': {'episode_meta': episodes_meta_dict[i], 'observation_meta': observation_meta.dict()},
                    'sequence': episodes_dict_nested[i]} for i in range(4)]

records_episodes[1]



/tmp/ipykernel_1117621/605212411.py:21: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.0.3/migration/
  'meta': {'episode_meta': episodes_meta_dict[i], 'observation_meta': observation_meta.dict()},


{'episodestart': 1689774112367182.0,
 'meta': {'episode_meta': {'vehicle': 'VB7',
   'driver': 'zheng-longfei',
   'episodestart': 1689774112367182.0},
  'observation_meta': {'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
     'thrust_unit_code': 'pct',
     'brake_unit_code': 'pct'},
    'state_number': 4,
    'unit_number': 4,
    'unit_duration': 1,
    'frequency': 50},
   'action_specs': {'action_unit_code': 'nm',
    'action_row_number': 4,
    'action_column_number': 17},
   'reward_specs': {'reward_unit': 'wh'},
   'site': 'anting'}},
 'sequence': [{'timestamp': 1689950512277535.0,
   'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
    'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
    'r2': [22.0, 23.0, 24.0, 25.0, 26.0],
    'speed': [40.0, 50.0, 60.0],
    'throttle': [0.0, 0.25, 0.5, 0.75, 1.0],
    'timestep': [1689946912377535.0, 1689946912397535.0, 1689946912417535.0]},
   'nstate': {'brake': [36.0, 37.0, 38.0, 39.0],
    'thrust': [32.0, 33.0, 34.0, 35.0],
  

# Write the episodes to a bag

In [14]:
os.getcwd()
os.chdir('bags')
os.getcwd()

'/home/n/devel/anting_remote/eos/data'

'/home/n/devel/anting_remote/eos/data/bags'

In [15]:
import dask.bag as db

b_episodes = db.from_sequence(records_episodes)
print(b_episodes.npartitions)
data = b_episodes.take(4,npartitions=4)
len(data)
b_episodes.to_avro('bag_episodes1.*.avro', schema=parsed_schema_episode)

4


4

['/home/n/devel/anting_remote/eos/data/bags/bag_episodes1.0.avro',
 '/home/n/devel/anting_remote/eos/data/bags/bag_episodes1.1.avro',
 '/home/n/devel/anting_remote/eos/data/bags/bag_episodes1.2.avro',
 '/home/n/devel/anting_remote/eos/data/bags/bag_episodes1.3.avro']

In [16]:
b_episodes_read = db.read_avro('bag_episodes1.*.avro')
rec = b_episodes_read.take(4,npartitions=4)
len(rec)
print(rec[0]['meta']['episode_meta']['vehicle'])
print(rec[0]['meta']['episode_meta']['driver'])


4

VB7
wang-cheng


# Add an episode to an existing bag

In [17]:
records_episode_to_add = [{'episodestart': episodes_meta_dict[4]['episodestart'],
                    'meta': {'episode_meta': episodes_meta_dict[4], 'observation_meta': observation_meta.model_dump()},
                    'sequence': episodes_dict_nested[4]}]
# records_episode_to_add

b_episodes_new = db.concat([b_episodes_read,
                            db.from_sequence(records_episode_to_add)])
print(b_episodes_new.npartitions)
b_episodes_new.to_avro('bag_episodes.*.avro', schema=parsed_schema_episode)

5


['/home/n/devel/anting_remote/eos/data/bags/bag_episodes.0.avro',
 '/home/n/devel/anting_remote/eos/data/bags/bag_episodes.1.avro',
 '/home/n/devel/anting_remote/eos/data/bags/bag_episodes.2.avro',
 '/home/n/devel/anting_remote/eos/data/bags/bag_episodes.3.avro',
 '/home/n/devel/anting_remote/eos/data/bags/bag_episodes.4.avro']

In [18]:
b_episodes_read = db.read_avro('bag_episodes.*.avro')
print(b_episodes_read.npartitions)
b_episodes_read.take(6,npartitions=6)

6


({'episodestart': 1689946912277535,
  'meta': {'episode_meta': {'vehicle': 'VB7',
    'driver': 'wang-cheng',
    'episodestart': 1689946912277535},
   'observation_meta': {'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
      'thrust_unit_code': 'pct',
      'brake_unit_code': 'pct'},
     'state_number': 4,
     'unit_number': 4,
     'unit_duration': 1,
     'frequency': 50},
    'action_specs': {'action_unit_code': 'nm',
     'action_row_number': 4,
     'action_column_number': 17},
    'reward_specs': {'reward_unit': 'wh'},
    'site': 'anting'}},
  'sequence': [{'timestamp': 1689950512277535,
    'state': {'velocity': [0.0, 1.0, 2.0, 3.0],
     'thrust': [4.0, 5.0, 6.0, 7.0],
     'brake': [8.0, 9.0, 10.0, 11.0],
     'timestep': [datetime.datetime(2023, 7, 21, 13, 41, 52, 277535, tzinfo=datetime.timezone.utc),
      datetime.datetime(2023, 7, 21, 13, 41, 52, 297535, tzinfo=datetime.timezone.utc),
      datetime.datetime(2023, 7, 21, 13, 41, 52, 317535, tzinfo=d

In [19]:
an_episode = b_episodes_read.take(2,npartitions=6)
type(an_episode[0])
len(an_episode)
ep_meta = an_episode[0]['meta']['episode_meta']
obs_meta = an_episode[0]['meta']['observation_meta']

obs_meta1 = an_episode[1]['meta']['observation_meta']
for key, val in (ep_meta | obs_meta).items():
    print(key, val)

obs_meta

dict

2

vehicle VB7
driver wang-cheng
episodestart 1689946912277535
state_specs {'state_unit_codes': {'velocity_unit_code': 'kph', 'thrust_unit_code': 'pct', 'brake_unit_code': 'pct'}, 'state_number': 4, 'unit_number': 4, 'unit_duration': 1, 'frequency': 50}
action_specs {'action_unit_code': 'nm', 'action_row_number': 4, 'action_column_number': 17}
reward_specs {'reward_unit': 'wh'}
site anting


{'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
   'thrust_unit_code': 'pct',
   'brake_unit_code': 'pct'},
  'state_number': 4,
  'unit_number': 4,
  'unit_duration': 1,
  'frequency': 50},
 'action_specs': {'action_unit_code': 'nm',
  'action_row_number': 4,
  'action_column_number': 17},
 'reward_specs': {'reward_unit': 'wh'},
 'site': 'anting'}

In [20]:
{**obs_meta}

{'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
   'thrust_unit_code': 'pct',
   'brake_unit_code': 'pct'},
  'state_number': 4,
  'unit_number': 4,
  'unit_duration': 1,
  'frequency': 50},
 'action_specs': {'action_unit_code': 'nm',
  'action_row_number': 4,
  'action_column_number': 17},
 'reward_specs': {'reward_unit': 'wh'},
 'site': 'anting'}

In [21]:
from eos.data_io.struct.eos_data import ObservationMeta

meta1 = ObservationMeta(**obs_meta1)
meta = ObservationMeta(**obs_meta)
meta1.have_same_meta(meta)
meta1.model_dump()

True

{'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
   'thrust_unit_code': 'pct',
   'brake_unit_code': 'pct'},
  'state_number': 4,
  'unit_number': 4,
  'unit_duration': 1,
  'frequency': 50},
 'action_specs': {'action_unit_code': 'nm',
  'action_row_number': 4,
  'action_column_number': 17},
 'reward_specs': {'reward_unit': 'wh'},
 'site': 'anting'}

In [22]:
b_episodes_read = db.read_avro('bag_episodes.*.avro')
print(b_episodes_read.npartitions)
b_episodes_read.take(6,npartitions=6)

6


({'episodestart': 1689946912277535,
  'meta': {'episode_meta': {'vehicle': 'VB7',
    'driver': 'wang-cheng',
    'episodestart': 1689946912277535},
   'observation_meta': {'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
      'thrust_unit_code': 'pct',
      'brake_unit_code': 'pct'},
     'state_number': 4,
     'unit_number': 4,
     'unit_duration': 1,
     'frequency': 50},
    'action_specs': {'action_unit_code': 'nm',
     'action_row_number': 4,
     'action_column_number': 17},
    'reward_specs': {'reward_unit': 'wh'},
    'site': 'anting'}},
  'sequence': [{'timestamp': 1689950512277535,
    'state': {'velocity': [0.0, 1.0, 2.0, 3.0],
     'thrust': [4.0, 5.0, 6.0, 7.0],
     'brake': [8.0, 9.0, 10.0, 11.0],
     'timestep': [datetime.datetime(2023, 7, 21, 13, 41, 52, 277535, tzinfo=datetime.timezone.utc),
      datetime.datetime(2023, 7, 21, 13, 41, 52, 297535, tzinfo=datetime.timezone.utc),
      datetime.datetime(2023, 7, 21, 13, 41, 52, 317535, tzinfo=d

# Deduplication

In [57]:
dicts = b_episodes_read.compute()
print(f"length: {len(dicts)}")
for b in dicts:
    b['episodestart'] = pd.to_datetime(b['episodestart'],unit='us')
    print(f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
        f"driver: {b['meta']['episode_meta']['driver']}; "
        f"episodestart: {b['episodestart']} ")

length: 6
vehicle: VB7; driver: wang-cheng; episodestart: 2023-07-21 13:41:52.277535 
vehicle: VB7; driver: zheng-longfei; episodestart: 2023-07-19 13:41:52.367182 
vehicle: MP73; driver: wang-cheng; episodestart: 2023-07-16 13:41:52.367182 
vehicle: VB7; driver: wang-cheng; episodestart: 2023-07-15 13:41:52.367182 
vehicle: MP73; driver: zheng-longfei; episodestart: 2023-07-14 13:41:52.367182 
vehicle: MP73; driver: zheng-longfei; episodestart: 2023-07-12 14:35:56.377754 


In [58]:

dicts = b_episodes_read.distinct(lambda x: (x['meta']['episode_meta']['driver'],
                                            x['meta']['episode_meta']['vehicle'],
                                            x['meta']['episode_meta']['episodestart'],)).compute()
for b in dicts:
    b['episodestart'] = pd.to_datetime(b['episodestart'],unit='us')
    print(f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
          f"driver: {b['meta']['episode_meta']['driver']}; "
          f"episodestart: {b['episodestart']} ")


vehicle: VB7; driver: wang-cheng; episodestart: 2023-07-21 13:41:52.277535 
vehicle: VB7; driver: zheng-longfei; episodestart: 2023-07-19 13:41:52.367182 
vehicle: MP73; driver: wang-cheng; episodestart: 2023-07-16 13:41:52.367182 
vehicle: VB7; driver: wang-cheng; episodestart: 2023-07-15 13:41:52.367182 
vehicle: MP73; driver: zheng-longfei; episodestart: 2023-07-14 13:41:52.367182 
vehicle: MP73; driver: zheng-longfei; episodestart: 2023-07-12 14:35:56.377754 


In [25]:
len(dicts)

6

# decode nested dicts to pandas dataframe

In [59]:
anep = dicts[0]
anep

{'episodestart': Timestamp('2023-07-21 13:41:52.277535'),
 'meta': {'episode_meta': {'vehicle': 'VB7',
   'driver': 'wang-cheng',
   'episodestart': 1689946912277535},
  'observation_meta': {'state_specs': {'state_unit_codes': {'velocity_unit_code': 'kph',
     'thrust_unit_code': 'pct',
     'brake_unit_code': 'pct'},
    'state_number': 4,
    'unit_number': 4,
    'unit_duration': 1,
    'frequency': 50},
   'action_specs': {'action_unit_code': 'nm',
    'action_row_number': 4,
    'action_column_number': 17},
   'reward_specs': {'reward_unit': 'wh'},
   'site': 'anting'}},
 'sequence': [{'timestamp': 1689950512277535,
   'state': {'velocity': [0.0, 1.0, 2.0, 3.0],
    'thrust': [4.0, 5.0, 6.0, 7.0],
    'brake': [8.0, 9.0, 10.0, 11.0],
    'timestep': [datetime.datetime(2023, 7, 21, 13, 41, 52, 277535, tzinfo=datetime.timezone.utc),
     datetime.datetime(2023, 7, 21, 13, 41, 52, 297535, tzinfo=datetime.timezone.utc),
     datetime.datetime(2023, 7, 21, 13, 41, 52, 317535, tzinfo=d

In [61]:
# anep['meta']['episode_meta']
# for k,v in anep['sequence']
len(anep['sequence'])
anep['sequence'][0]

5

{'timestamp': 1689950512277535,
 'state': {'velocity': [0.0, 1.0, 2.0, 3.0],
  'thrust': [4.0, 5.0, 6.0, 7.0],
  'brake': [8.0, 9.0, 10.0, 11.0],
  'timestep': [datetime.datetime(2023, 7, 21, 13, 41, 52, 277535, tzinfo=datetime.timezone.utc),
   datetime.datetime(2023, 7, 21, 13, 41, 52, 297535, tzinfo=datetime.timezone.utc),
   datetime.datetime(2023, 7, 21, 13, 41, 52, 317535, tzinfo=datetime.timezone.utc),
   datetime.datetime(2023, 7, 21, 13, 41, 52, 337535, tzinfo=datetime.timezone.utc)]},
 'action': {'r0': [12.0, 13.0, 14.0, 15.0, 16.0],
  'r1': [17.0, 18.0, 19.0, 20.0, 21.0],
  'r2': [22.0, 23.0, 24.0, 25.0, 26.0],
  'speed': [40.0, 50.0, 60.0],
  'throttle': [0.0, 0.25, 0.5, 0.75, 1.0],
  'timestep': [datetime.datetime(2023, 7, 21, 13, 41, 52, 377535, tzinfo=datetime.timezone.utc),
   datetime.datetime(2023, 7, 21, 13, 41, 52, 397535, tzinfo=datetime.timezone.utc),
   datetime.datetime(2023, 7, 21, 13, 41, 52, 417535, tzinfo=datetime.timezone.utc)]},
 'reward': {'work': [27.0],

In [62]:
# dict_observations = {(meta['vehicle'], meta['driver'], meta['episodestart'],
#                       pd.to_datetime(obs['timestamp'],unit='us'), qtuple): value
#                         for meta, obs in zip(anep['meta']['episode_meta'],anep['sequence'])
#                         for qtuple, value in obs.items() if qtuple != 'timestamp'}

dict_observations = [
    {
        (anep['meta']['episode_meta']['vehicle'],
            anep['meta']['episode_meta']['driver'],
            pd.to_datetime(anep['meta']['episode_meta']['episodestart'],unit='us'),
            pd.to_datetime(step['timestamp'],unit='us'),
            qtuple,
            rows,
         	idx
        ): item
        for qtuple, obs in step.items() if qtuple != 'timestamp'
        for rows, value in obs.items()
    	for idx,item in enumerate(value)
    }
    for step in anep['sequence']
]

dict_ep = {k:v for d in dict_observations for k,v in d.items()}


# d = dict_observations[0]
ser_decoded = pd.Series(dict_ep)
ser_decoded.index.names = ['vehicle', 'driver', 'episodestart', 'timestamp', 'qtuple', 'rows', 'idx']
df_decoded = ser_decoded.unstack(level=['qtuple', 'rows', 'idx'])
df_decoded

qtuple                                                                      state  \
rows                                                                     velocity   
idx                                                                             0   
vehicle driver     episodestart               timestamp                             
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535      0.0   
                                              2023-07-21 15:41:52.277535      0.0   
                                              2023-07-21 16:41:52.277535      0.0   
                                              2023-07-21 17:41:52.277535      0.0   
                                              2023-07-21 18:41:52.277535      0.0   

qtuple                                                                         \
rows                                                                            
idx                                                                         1   
vehicle driver     episodestart               timestamp                         
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  1.0   
                                              2023-07-21 15:41:52.277535  1.0   
                                              2023-07-21 16:41:52.277535  1.0   
                                              2023-07-21 17:41:52.277535  1.0   
                                              2023-07-21 18:41:52.277535  1.0   

qtuple                                                                         \
rows                                                                            
idx                                                                         2   
vehicle driver     episodestart               timestamp                         
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  2.0   
                                              2023-07-21 15:41:52.277535  2.0   
                                              2023-07-21 16:41:52.277535  2.0   
                                              2023-07-21 17:41:52.277535  2.0   
                                              2023-07-21 18:41:52.277535  2.0   

qtuple                                                                         \
rows                                                                            
idx                                                                         3   
vehicle driver     episodestart               timestamp                         
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  3.0   
                                              2023-07-21 15:41:52.277535  3.0   
                                              2023-07-21 16:41:52.277535  3.0   
                                              2023-07-21 17:41:52.277535  3.0   
                                              2023-07-21 18:41:52.277535  3.0   

qtuple                                                                           \
rows                                                                     thrust   
idx                                                                           0   
vehicle driver     episodestart               timestamp                           
VB7     wang-cheng 2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535    4.0   
                                              2023-07-21 15:41:52.277535    4.0   
                                              2023-07-21 16:41:52.277535    4.0   
                                              2023-07-21 17:41:52.277535    4.0   
                                              2023-07-21 18:41:52.277535    4.0   

qtuple                                                                         \
rows                                                                            
idx                                                                         1   
vehicle driver     episodestart               timesta

In [63]:
len(dicts)

6

In [64]:
from functools import reduce

df_episodes_list = []
for ep in dicts:
    dict_observations = [
        {
            (ep['meta']['episode_meta']['vehicle'],
                ep['meta']['episode_meta']['driver'],
                pd.to_datetime(ep['meta']['episode_meta']['episodestart'],unit='us'),
                pd.to_datetime(step['timestamp'],unit='us'),
                qtuple,
                rows,
                idx
            ): item
            for qtuple, obs in step.items() if qtuple != 'timestamp'
            for rows, value in obs.items()
            for idx,item in enumerate(value)
        }
        for step in ep['sequence']
    ]

    dict_ep = {k:v for d in dict_observations for k,v in d.items()}

    ser_decoded = pd.Series(dict_ep)
    ser_decoded.index.names = ['vehicle', 'driver', 'episodestart', 'timestamp', 'qtuple', 'rows', 'idx']
    df_decoded = ser_decoded.unstack(level=['qtuple', 'rows', 'idx'])
    df_episodes_list.append(df_decoded)
    # df_decoded

try:
    df_episodes = reduce(
        lambda left,right: pd.concat([left,right], axis=0, ignore_index=False),df_episodes_list
    )
except Exception as e:
    print(e)

df_episodes.sort_index(inplace=True)
df_episodes

qtuple                                                                         state  \
rows                                                                        velocity   
idx                                                                                0   
vehicle driver        episodestart               timestamp                             
MP73    wang-cheng    2023-07-16 13:41:52.367182 2023-07-21 15:41:52.277535      0.0   
                                                 2023-07-21 18:41:52.277535      0.0   
        zheng-longfei 2023-07-12 14:35:56.377754 2023-07-19 16:35:56.287255      0.0   
                                                 2023-07-19 17:35:56.287255      0.0   
                                                 2023-07-19 18:35:56.287255      0.0   
                                                 2023-07-19 19:35:56.287255      0.0   
                      2023-07-14 13:41:52.367182 2023-07-21 15:41:52.277535      0.0   
                                                 2023-07-21 18:41:52.277535      0.0   
VB7     wang-cheng    2023-07-15 13:41:52.367182 2023-07-21 14:41:52.277535      0.0   
                                                 2023-07-21 16:41:52.277535      0.0   
                                                 2023-07-21 17:41:52.277535      0.0   
                      2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535      0.0   
                                                 2023-07-21 15:41:52.277535      0.0   
                                                 2023-07-21 16:41:52.277535      0.0   
                                                 2023-07-21 17:41:52.277535      0.0   
                                                 2023-07-21 18:41:52.277535      0.0   
        zheng-longfei 2023-07-19 13:41:52.367182 2023-07-21 14:41:52.277535      0.0   
                                                 2023-07-21 15:41:52.277535      0.0   
                                                 2023-07-21 17:41:52.277535      0.0   
                                                 2023-07-21 18:41:52.277535      0.0   

qtuple                                                                            \
rows                                                                               
idx                                                                            1   
vehicle driver        episodestart               timestamp                         
MP73    wang-cheng    2023-07-16 13:41:52.367182 2023-07-21 15:41:52.277535  1.0   
                                                 2023-07-21 18:41:52.277535  1.0   
        zheng-longfei 2023-07-12 14:35:56.377754 2023-07-19 16:35:56.287255  1.0   
                                                 2023-07-19 17:35:56.287255  1.0   
                                                 2023-07-19 18:35:56.287255  1.0   
                                                 2023-07-19 19:35:56.287255  1.0   
                      2023-07-14 13:41:52.367182 2023-07-21 15:41:52.277535  1.0   
                                                 2023-07-21 18:41:52.277535  1.0   
VB7     wang-cheng    2023-07-15 13:41:52.367182 2023-07-21 14:41:52.277535  1.0   
                                                 2023-07-21 16:41:52.277535  1.0   
                                                 2023-07-21 17:41:52.277535  1.0   
                      2023-07-21 13:41:52.277535 2023-07-21 14:41:52.277535  1.0   
                                                 2023-07-21 15:41:52.277535  1.0   
                                                 2023-07-21 16:41:52.277535  1.0   
                                                 2023-07-21 17:41:52.277535  1.0   
                                                 2023-07-21 18:41:52.277535  1.0   
        zheng-longfei 2023-07-19 13:41:52.367182 2023-07-21 14:41:52.277535  1.0   
                                                 2023-07-21 15:41:52.277535  1.0   
                                       

In [65]:

len(df_episodes_list)

6

In [66]:
from keras.preprocessing.sequence import pad_sequences
idx = pd.IndexSlice
episodestart_index = df_episodes.index.unique(level='episodestart')
# episodestart_index

# array of rewards for minibatch
batch = df_episodes.sort_index(inplace=False, axis=1)
# idx = pd.IndexSlice
df_rewards = batch.loc[
    :, idx['reward', 'work']
]
rewards_list = [
    df_rewards.loc[idx[:, :, ep_start, :]].values.tolist()
    for ep_start in episodestart_index
]
r_n_t = pad_sequences(
    rewards_list, padding='post', dtype=np.float32, value=-10000.0
)
print(f'r_n_t: {rewards_list} {r_n_t.shape}')

# array of states for minibatch
df_states = batch.loc[
    :, idx['state', ['velocity', 'thrust', 'brake']]  # type: ignore
]  # same order as inference !!!
states_list = [
    df_states.loc[idx[:, :, ep_start, :]].values.tolist()  # type: ignore
    for ep_start in episodestart_index
]
s_n_t = pad_sequences(
    states_list, padding='post', dtype=np.float32, value=-10000.0
)
print(f's_n_t: {states_list} {s_n_t.shape}')

# array of actions for minibatch
df_actions = batch.loc[:, idx['action', torque_table_row_names]]  # type: ignore
actions_list = [
    df_actions.loc[idx[:, :, ep_start, :]].values.tolist()  # type: ignore
    for ep_start in episodestart_index
]
a_n_t = pad_sequences(
    actions_list, padding='post', dtype=np.float32, value=-10000.0
)
print(f'a_n_t: {actions_list} {a_n_t.shape}')

# array of next_states for minibatch
df_nstates = batch.loc[:, idx['nstate', ['velocity', 'thrust', 'brake']]]  # type: ignore
nstates_list = [
    df_nstates.loc[idx[:, :, ep_start, :]].values.tolist()  # type: ignore
    for ep_start in episodestart_index
]
ns_n_t = pad_sequences(
    nstates_list, padding='post', dtype=np.float32, value=-10000.0
)

print(f'ns_n_t: {nstates_list} {ns_n_t.shape}')


r_n_t: [[[27.0], [27.0]], [[27.0], [27.0], [27.0], [27.0]], [[27.0], [27.0]], [[27.0], [27.0], [27.0]], [[27.0], [27.0], [27.0], [27.0], [27.0]], [[27.0], [27.0], [27.0], [27.0]]] (6, 5, 1)
s_n_t: [[[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]], [[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]], [[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]], [[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]], [[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0], [0.0, 1.0, 2.0, 3.0, 4.0,

In [67]:
r_n_t
a_n_t
s_n_t
ns_n_t

array([[[    27.],
        [    27.],
        [-10000.],
        [-10000.],
        [-10000.]],

       [[    27.],
        [    27.],
        [    27.],
        [    27.],
        [-10000.]],

       [[    27.],
        [    27.],
        [-10000.],
        [-10000.],
        [-10000.]],

       [[    27.],
        [    27.],
        [    27.],
        [-10000.],
        [-10000.]],

       [[    27.],
        [    27.],
        [    27.],
        [    27.],
        [    27.]],

       [[    27.],
        [    27.],
        [    27.],
        [    27.],
        [-10000.]]], dtype=float32)

array([[[    12.,     13.,     14.,     15.,     16.,     17.,     18.,
             19.,     20.,     21.,     22.,     23.,     24.,     25.,
             26.],
        [    12.,     13.,     14.,     15.,     16.,     17.,     18.,
             19.,     20.,     21.,     22.,     23.,     24.,     25.,
             26.],
        [-10000., -10000., -10000., -10000., -10000., -10000., -10000.,
         -10000., -10000., -10000., -10000., -10000., -10000., -10000.,
         -10000.],
        [-10000., -10000., -10000., -10000., -10000., -10000., -10000.,
         -10000., -10000., -10000., -10000., -10000., -10000., -10000.,
         -10000.],
        [-10000., -10000., -10000., -10000., -10000., -10000., -10000.,
         -10000., -10000., -10000., -10000., -10000., -10000., -10000.,
         -10000.]],

       [[    12.,     13.,     14.,     15.,     16.,     17.,     18.,
             19.,     20.,     21.,     22.,     23.,     24.,     25.,
             26.],
        [    12.,   

array([[[ 0.0e+00,  1.0e+00,  2.0e+00,  3.0e+00,  4.0e+00,  5.0e+00,
          6.0e+00,  7.0e+00,  8.0e+00,  9.0e+00,  1.0e+01,  1.1e+01],
        [ 0.0e+00,  1.0e+00,  2.0e+00,  3.0e+00,  4.0e+00,  5.0e+00,
          6.0e+00,  7.0e+00,  8.0e+00,  9.0e+00,  1.0e+01,  1.1e+01],
        [-1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04,
         -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04],
        [-1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04,
         -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04],
        [-1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04,
         -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04, -1.0e+04]],

       [[ 0.0e+00,  1.0e+00,  2.0e+00,  3.0e+00,  4.0e+00,  5.0e+00,
          6.0e+00,  7.0e+00,  8.0e+00,  9.0e+00,  1.0e+01,  1.1e+01],
        [ 0.0e+00,  1.0e+00,  2.0e+00,  3.0e+00,  4.0e+00,  5.0e+00,
          6.0e+00,  7.0e+00,  8.0e+00,  9.0e+00,  1.0e+01,  1.1e+01],
        [ 0.0e+00,  1.0e+

array([[[    28.,     29.,     30.,     31.,     32.,     33.,     34.,
             35.,     36.,     37.,     38.,     39.],
        [    28.,     29.,     30.,     31.,     32.,     33.,     34.,
             35.,     36.,     37.,     38.,     39.],
        [-10000., -10000., -10000., -10000., -10000., -10000., -10000.,
         -10000., -10000., -10000., -10000., -10000.],
        [-10000., -10000., -10000., -10000., -10000., -10000., -10000.,
         -10000., -10000., -10000., -10000., -10000.],
        [-10000., -10000., -10000., -10000., -10000., -10000., -10000.,
         -10000., -10000., -10000., -10000., -10000.]],

       [[    28.,     29.,     30.,     31.,     32.,     33.,     34.,
             35.,     36.,     37.,     38.,     39.],
        [    28.,     29.,     30.,     31.,     32.,     33.,     34.,
             35.,     36.,     37.,     38.,     39.],
        [    28.,     29.,     30.,     31.,     32.,     33.,     34.,
             35.,     36.,     37.,  

# sample

In [33]:
from dask.bag import random
print(f"partitions: {b_episodes_read.npartitions}")
# bag = random.choices(b_episodes_read,16, split_every=4)
bag = random.sample(b_episodes_read,3, split_every=6)
print(f"partitions: {bag.npartitions}")
dicts = bag.compute()
print(f"length: {len(dicts)}")
for b in dicts:
    b['episodestart'] = pd.to_datetime(b['episodestart'],unit='us')
    print(f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
        f"driver: {b['meta']['episode_meta']['driver']}; "
        f"episodestart: {b['episodestart']} ")

partitions: 6
partitions: 1
length: 3
vehicle: MP73; driver: zheng-longfei; episodestart: 2023-07-14 13:41:52.367182 
vehicle: VB7; driver: wang-cheng; episodestart: 2023-07-21 13:41:52.277535 
vehicle: MP73; driver: zheng-longfei; episodestart: 2023-07-12 14:35:56.377754 


In [34]:
from eos.utils.eos_pandas import avro_ep_decoding

In [35]:
df_episodes_list = avro_ep_decoding(bag.compute())
try:
    df_episodes = reduce(
        lambda left,right: pd.concat([left,right], axis=0, ignore_index=False),df_episodes_list
    )
except Exception as e:
    print(e)

df_episodes.sort_index(inplace=True)
df_episodes

qtuple                                                                         state  \
rows                                                                        velocity   
idx                                                                                0   
vehicle driver        episodestart               timestamp                             
MP73    wang-cheng    2023-07-16 13:41:52.367182 2023-07-21 15:41:52.277535      0.0   
                                                 2023-07-21 18:41:52.277535      0.0   
VB7     wang-cheng    2023-07-15 13:41:52.367182 2023-07-21 14:41:52.277535      0.0   
                                                 2023-07-21 16:41:52.277535      0.0   
                                                 2023-07-21 17:41:52.277535      0.0   
        zheng-longfei 2023-07-19 13:41:52.367182 2023-07-21 14:41:52.277535      0.0   
                                                 2023-07-21 15:41:52.277535      0.0   
                                                 2023-07-21 17:41:52.277535      0.0   
                                                 2023-07-21 18:41:52.277535      0.0   

qtuple                                                                            \
rows                                                                               
idx                                                                            1   
vehicle driver        episodestart               timestamp                         
MP73    wang-cheng    2023-07-16 13:41:52.367182 2023-07-21 15:41:52.277535  1.0   
                                                 2023-07-21 18:41:52.277535  1.0   
VB7     wang-cheng    2023-07-15 13:41:52.367182 2023-07-21 14:41:52.277535  1.0   
                                                 2023-07-21 16:41:52.277535  1.0   
                                                 2023-07-21 17:41:52.277535  1.0   
        zheng-longfei 2023-07-19 13:41:52.367182 2023-07-21 14:41:52.277535  1.0   
                                                 2023-07-21 15:41:52.277535  1.0   
                                                 2023-07-21 17:41:52.277535  1.0   
                                                 2023-07-21 18:41:52.277535  1.0   

qtuple                                                                            \
rows                                                                               
idx                                                                            2   
vehicle driver        episodestart               timestamp                         
MP73    wang-cheng    2023-07-16 13:41:52.367182 2023-07-21 15:41:52.277535  2.0   
                                                 2023-07-21 18:41:52.277535  2.0   
VB7     wang-cheng    2023-07-15 13:41:52.367182 2023-07-21 14:41:52.277535  2.0   
                                                 2023-07-21 16:41:52.277535  2.0   
                                                 2023-07-21 17:41:52.277535  2.0   
        zheng-longfei 2023-07-19 13:41:52.367182 2023-07-21 14:41:52.277535  2.0   
                                                 2023-07-21 15:41:52.277535  2.0   
                                                 2023-07-21 17:41:52.277535  2.0   
                                                 2023-07-21 18:41:52.277535  2.0   

qtuple                                                                            \
rows                                                                               
idx                                                                            3   
vehicle driver        episodestart               timestamp                         
MP73    wang-cheng    2023-07-16 13:41:52.367182 2023-07-21 15:41:52.277535  3.0   
                                                 2023-07-21 18:41:52.277535  3.0   
VB7     wang-cheng    2023-07-15 13:41:52.367182 2023-07-21 14:41:52.277535  3.0   
                                                 2023-07-21 16:41:52.277535  3.0 

In [36]:
dicts = b_episodes_read.compute()
print(f"length: {len(dicts)}")
for b in dicts:
    b['episodestart'] = pd.to_datetime(b['episodestart'],unit='us')
    print(f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
        f"driver: {b['meta']['episode_meta']['driver']}; "
        f"episodestart: {b['episodestart']} ")

length: 6
vehicle: VB7; driver: wang-cheng; episodestart: 2023-07-21 13:41:52.277535 
vehicle: VB7; driver: zheng-longfei; episodestart: 2023-07-19 13:41:52.367182 
vehicle: MP73; driver: wang-cheng; episodestart: 2023-07-16 13:41:52.367182 
vehicle: VB7; driver: wang-cheng; episodestart: 2023-07-15 13:41:52.367182 
vehicle: MP73; driver: zheng-longfei; episodestart: 2023-07-14 13:41:52.367182 
vehicle: MP73; driver: zheng-longfei; episodestart: 2023-07-12 14:35:56.377754 


In [37]:
from pandas.tseries.offsets import DateOffset
ts0 = ts - DateOffset(days=5, hours=1)
ts1 = ts + DateOffset(hours=1)
type(ts)
print(ts0, ts1)
ts0.timestamp()
ts0

pandas._libs.tslibs.timestamps.Timestamp

2023-07-16 12:41:52.277535 2023-07-21 14:41:52.277535


1689511312.277535

Timestamp('2023-07-16 12:41:52.277535')

In [38]:
queried = b_episodes_read.filter(lambda x: pd.to_datetime(x['meta']['episode_meta']['episodestart'],unit='us') > ts0)
# bag = random.choices(queried,7)
print(f"ts0:{ts0}")
dicts = queried.compute()
for b in dicts:
    b['episodestart'] = pd.to_datetime(b['episodestart'],unit='us')
    # pd.to_datetime(b['episodestart'],unit='us')
    print(f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
        f"driver: {b['meta']['episode_meta']['driver']}; "
        f"episodestart: {b['episodestart']} ")

ts0:2023-07-16 12:41:52.277535
vehicle: VB7; driver: wang-cheng; episodestart: 2023-07-21 13:41:52.277535 
vehicle: VB7; driver: zheng-longfei; episodestart: 2023-07-19 13:41:52.367182 
vehicle: MP73; driver: wang-cheng; episodestart: 2023-07-16 13:41:52.367182 


In [39]:

# queried = b_episodes_read.filter(lambda x: x['meta']['episode_meta']['vehicle'] == 'VB7' and
#                                         x['meta']['episode_meta']['driver'] == 'wang-cheng' and
#                                         pd.to_datetime(x['meta']['episode_meta']['episodestart'],unit='us') > ts0)

print(ts0, ts1)
queried = b_episodes_read.filter(lambda x: x['meta']['episode_meta']['driver'] == 'wang-cheng' and
                        ts0 < pd.to_datetime(x['meta']['episode_meta']['episodestart'], unit='us') < ts1)
# bag = random.choices(queried,7)
dicts = queried.compute()
for b in dicts:
    b['episodestart'] = pd.to_datetime(b['episodestart'],unit='us')
    # pd.to_datetime(b['episodestart'],unit='us')
    print(f"vehicle: {b['meta']['episode_meta']['vehicle']}; "
        f"driver: {b['meta']['episode_meta']['driver']}; "
        f"episodestart: {b['episodestart']} ")

2023-07-16 12:41:52.277535 2023-07-21 14:41:52.277535
vehicle: VB7; driver: wang-cheng; episodestart: 2023-07-21 13:41:52.277535 
vehicle: MP73; driver: wang-cheng; episodestart: 2023-07-16 13:41:52.367182 


In [40]:
import dask.bag as db
a = db.from_sequence([1, 2, 3])
b = db.from_sequence([4, 5, 6, 1, 2, 3])
c = db.concat([a, b])
c.compute()
c.distinct().compute()

[1, 2, 3, 4, 5, 6, 1, 2, 3]

[1, 2, 3, 4, 5, 6]